#### This notebook is just to show how rag can be useful, we will just feed data as text and will observe the answers to see that when context is provided answers are much relevant . No optimization have been done , default data load and using ollama2 as model

In [39]:
from llama_index.readers.database import DatabaseReader
from llama_index.core import VectorStoreIndex
from dotenv import load_dotenv
import os

In [40]:
# Load the environment variables
load_dotenv('..\.env')

True

In [41]:
# Initialize DatabaseReader object with the following parameters:

db = DatabaseReader(
    scheme="postgresql",  # Database Scheme
    host=os.getenv("DB_HOST"),  # Database Host
    port=os.getenv("DB_PORT"),  # Database Port
    user=os.getenv("DB_USER"),  # Database User
    password=os.getenv("DB_PASSWORD"),  # Database Password
    dbname=os.getenv("DB_NAME"),  # Database Name
)

In [42]:
### DatabaseReader class ###
# db is an instance of DatabaseReader:
print(type(db))
# DatabaseReader available method:
print(type(db.load_data))

### SQLDatabase class ###
# db.sql is an instance of SQLDatabase:
print(type(db.sql_database))
# SQLDatabase available methods:
print(type(db.sql_database.from_uri))
print(type(db.sql_database.get_single_table_info))
print(type(db.sql_database.get_table_columns))
print(type(db.sql_database.get_usable_table_names))
print(type(db.sql_database.insert_into_table))
print(type(db.sql_database.run_sql))
# SQLDatabase available properties:
print(type(db.sql_database.dialect))
print(type(db.sql_database.engine))

<class 'llama_index.readers.database.base.DatabaseReader'>
<class 'method'>
<class 'llama_index.core.utilities.sql_wrapper.SQLDatabase'>
<class 'method'>
<class 'method'>
<class 'method'>
<class 'method'>
<class 'method'>
<class 'method'>
<class 'str'>
<class 'sqlalchemy.engine.base.Engine'>


In [43]:
# The below SQL Query example returns comments from subreddit indiacoffee posts

query = f"""
    SELECT
        body AS text
    FROM public.indiacoffee_comments
    """

In [47]:
texts = db.sql_database.run_sql(command=query)

In [48]:
print(type(texts))

<class 'tuple'>


In [50]:
print(texts[0][:1000])

[('Nice article. This is my regular coffee now, since they started selling it. I was taken aback after the first sip kinda gave me strawberry notes. I never thought some coffee in India could produce such wonderful tasting notes.',), ('Exactly! Just love it. I finished it under 7 days. Today is the last day.',), (" I just hope they don't take it off the list. This is better than most AAA coffee sold in India, In my opinion",), ("Hey I am trying to get into brewing coffee and leave the instant coffee what do you recommend for my first try of blue tokai coffee (thinking of buying a Channi grind as I don't have any special equipment)",), ('So happy to hear this. You should totally go for Channi grind although you can even try the easy pour satchels. \n\nI would recommend going for Attiknan Estate, since it has balanced flavours. \n\nDrinking and making specialty coffee for this first time, can be a very frustrating and confusing...',), ('Thinking of going for https://bluetokaicoffee.com/c

In [52]:
documents = db.load_data(query=query)

In [53]:
print(type(documents))

<class 'list'>


In [54]:
# print(documents)

In [1]:
# creating embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

c:\Users\snehi\Desktop\Learning\.dev_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(Settings.chunk_size)
print(Settings.chunk_overlap)

1024
200


In [ ]:
# can change chunk_size and chunk_overlap
# Settings.chunk_size = 512
# Settings.chunk_overlap = 50

In [17]:
# creating index
index = VectorStoreIndex.from_documents(documents)

In [22]:
# storing index in folder
index.storage_context.persist(persist_dir="embed_dir/")

In [57]:
# reloading index

from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="embed_dir/")

# load index
index = load_index_from_storage(storage_context)

In [58]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.llms.ollama import Ollama

# ollama
Settings.llm = Ollama(model="llama2", request_timeout=60.0)

### `output without context - the responses are not at all relevant to indian coffee community discussions`

In [60]:
llm_without_context = Ollama(model="llama2", request_timeout=60.0)
resp = llm_without_context.complete("what are some best coffee roasters?")
print(resp)


There are many great coffee roasters out there, and the best one for you will depend on your personal preferences and tastes. Here are some highly-regarded coffee roasters that you may want to consider:

1. Intelligentsia Coffee Roasters (Chicago, IL): Known for their high-quality, artisanal coffee beans and extensive knowledge of the coffee industry.
2. Stumptown Coffee Roasters (Portland, OR): A popular roaster with a strong reputation for their direct trade practices and unique blends.
3. Blue Bottle Coffee (Oakland, CA): This small-batch roaster is known for their carefully sourced, expertly roasted beans and modern coffee bars.
4. Four Barrel Coffee (San Francisco, CA): A popular roaster with a strong commitment to sustainability and ethical sourcing, as well as delicious coffee.
5. La Colombe Coffee Roasters (Philadelphia, PA): This roaster has gained a following for their ethically sourced, expertly roasted beans and cafes in multiple locations.
6. Toby's Estate Coffee Roasters

In [62]:
resp = llm_without_context.complete("what are best blends from Savour Works roasters?")
print(resp)


Savour Works Roasters is a well-known specialty coffee roaster based in Toronto, Canada. They offer a variety of high-quality coffee blends that are crafted to showcase the unique flavor profiles of each coffee bean. Here are some of the best blends from Savour Works Roasters:

1. Ethiopian Yirgacheffe: This is a classic blend that features bright acidity and floral notes, with hints of citrus and berries. It's a great choice for those who prefer a more delicate and refined cup of coffee.
2. Brazilian Fazenda: This blend is a rich and full-bodied coffee with notes of chocolate, caramel, and nuts. It's roasted to perfection to bring out the sweet and smooth flavors that Brazilian coffee is known for.
3. Kenyan Gethumbi: This blend has a bright and complex flavor profile with notes of fruit, floral, and tea-like flavors. It's a great choice for those who enjoy a more acidic and complex cup of coffee.
4. Colombian Supremo: This blend is a medium-bodied coffee with a smooth and balanced f

In [68]:
resp = llm_without_context.complete("what are good blends from devan roasters?")
print(resp)

Devan Roasters is a well-known coffee roastery based in India, and they offer a wide range of delicious blends. Here are some popular blends from Devan Roasters that you might want to try:

1. Devan's Delight: A classic blend of Indian and African coffee beans, with notes of chocolate and caramel.
2. Coffee Fusion: A unique blend of Ethiopian Yirgacheffe, Brazilian Santos, and Indonesian Sumatran coffee beans, resulting in a smooth and well-balanced cup.
3. Decaffeinated Delight: A decaf version of Devan's Delight, perfect for those who want to enjoy the rich flavors of Indian and African coffee without the caffeine kick.
4. Espresso Roast: A dark roast blend of Arabica beans from India, Ethiopia, and Brazil, with a bold and intense flavor profile.
5. French Roast: A smoky and rich blend of Arabica beans from India, Vietnam, and Indonesia, with a deep, smooth flavor and a slightly sweet aftertaste.
6. Costa Rican Tarrazu: A medium roast blend of Arabica beans from Costa Rica, with a ba

### `output with context  - the responses reflects the community discussions`

In [59]:
query_engine = index.as_query_engine()
response = query_engine.query("what are some best coffee roasters?")
print(response)

There are several excellent coffee roasters available, each offering unique flavor profiles and roasting techniques. To name a few, you may want to consider:

1. Subko: Known for their high-quality, small-batch roasts, Subko offers a range of darker roasts with notes of caramel, toffee, and bitter chocolate.
2. qbf: This coffee roaster is particularly known for its medium to dark roasts, with a focus on fruity flavors like stone fruits and cherries.
3. Savour Works: Offering a variety of roast levels, from light to dark, Savour Works is a great option for those who prefer a more balanced taste.
4. Araku: This coffee roaster is known for its organic and fair-trade practices, as well as its unique blends with flavors like chocolate and nuts.
5. Naivo: With a focus on small-batch, specialty coffee, Naivo offers a range of darker roasts with notes of caramel, toffee, and bitter chocolate.
6. Half Light: This coffee roaster is known for its medium roasts with a smooth, balanced taste and a 

In [61]:
response = query_engine.query("what are best blends from Savour Works roasters?")
print(response)

The best blends from Savorworks roasters are unquestionably their signature Fruits Bomb. This coffee is a must-try for anyone looking for a fruity and top-tier coffee experience. Additionally, the medium-light roasts offered by Fraction9 are also highly recommended, particularly their Breakfast Roast and Sweet Roast.


In [67]:
response = query_engine.query("what are good blends from devan roasters?")
print(response)

Devans is a traditional second wave coffee roaster known for offering good-quality coffee at an affordable price. While their coffees may not be the highest quality, they are still a great value for money, especially if you prefer darker roasts. Based on the context information provided, it seems that Devans is a good brand for those who enjoy darker roasts. However, personally, the speaker prefers light to medium roasts and buys from other brands such as Savorworks, Blue Tokai, Bloom Coffee, and others. Additionally, the speaker recommends buying directly from the roaster rather than through Amazon.


In [75]:
response = query_engine.query("which coffee grinder to buy for good grind from the context provided?")
print(response)

Based on the context provided, it is recommended to purchase the Timemore C2 grinder. This grinder is considered a great value for money and is suitable for those who are looking for a good grind without breaking the bank. Additionally, if you cannot afford the Timemore C2, the cheap burr grinder from Amazon is also suggested as it provides high-end specs at an affordable price. However, it is important to note that the Aeropress is also recommended as it is a forgiving coffee equipment that is relatively simple to use and can produce decent cups of coffee. Ultimately, the choice of grinder will depend on your personal preferences and budget.
